# test_00_sync

In [ ]:
#|default_exp test_00_sync
#|export_as_func true

In [ ]:
#|hide
import nblite; from nblite import show_doc; nblite.nbl_export()
import tests as this_module

Environment variable DISABLE_NBLITE_EXPORT is set to True, skipping export.


In [ ]:
#|top_export
import subprocess
from pathlib import Path
import shutil
import toml

from repoyard import const
from repoyard.cmds import *
from repoyard._repos import get_repoyard_meta
from repoyard.config import get_config

In [ ]:
#|set_func_signature
def test_00_sync(): ...

Parameters

In [ ]:
#|export
num_test_repos = 20

# Initialise `init_repoyard`

In [ ]:
#|export
# Set up test folders
import tempfile
test_folder_path = Path(tempfile.mkdtemp(prefix="sync_repo", dir="/tmp"))
test_folder_path.mkdir(parents=True, exist_ok=True)
config_path = test_folder_path / ".config" / "repoyard" / "config.toml"
data_path = test_folder_path / ".repoyard"

# Run init
init_repoyard(config_path=config_path, data_path=data_path)

# Add a storage location 'my_remote'
import toml
test_storage_location_name = "my_remote"
config_dump = toml.load(config_path)
remote_rclone_path = Path(tempfile.mkdtemp(prefix="rclone_remote", dir="/tmp"))
config_dump['storage_locations'][test_storage_location_name] = {
    'storage_type' : "rclone",
    'store_path' : "repoyard",
}
config_path.write_text(toml.dumps(config_dump))

# Load config
config = get_config(config_path)

# Set up a rclone remote path for testing
config.rclone_config_path.write_text(f"""
[{test_storage_location_name}]
type = alias
remote = {remote_rclone_path}
""");

Using a non-default config path. Please set the environment variable REPOYARD_CONFIG_PATH to the given config path for repoyard to use it. 
Creating config file at: /tmp/sync_repokfnhpk9u/.config/repoyard/config.toml
Creating folder: /tmp/sync_repokfnhpk9u/.repoyard
Creating folder: /tmp/sync_repokfnhpk9u/.repoyard/local_store
Creating rclone config file at: /tmp/sync_repokfnhpk9u/.config/repoyard/repoyard_rclone.conf
Done!

You can modify the config at: /tmp/sync_repokfnhpk9u/.config/repoyard/config.toml
All repoyard data is stored in: /tmp/sync_repokfnhpk9u/.repoyard


# Create some repos using `new_repo` and sync them using `sync_repo`

In [ ]:
#|export
repo_full_names = []
for i in range(num_test_repos):
    repo_full_name = new_repo(config_path=config_path, repo_name=f"test_repo_{i}", storage_location=test_storage_location_name)
    repo_full_names.append(repo_full_name)
    
# Verify that the repos are included
repoyard_meta = get_repoyard_meta(config, force_create=True)
for repo_full_name in repo_full_names:
    assert repoyard_meta.by_full_name[repo_full_name].check_included(config)

Initialized empty Git repository in /private/tmp/sync_repokfnhpk9u/.repoyard/local_store/my_remote/01K9YZXGBTNWA3DAFPFPNMPYT7__test_repo_0/data/.git/
Initialized empty Git repository in /private/tmp/sync_repokfnhpk9u/.repoyard/local_store/my_remote/01K9YZXGCDT7FNF9J7ZH1F4M6N__test_repo_1/data/.git/
Initialized empty Git repository in /private/tmp/sync_repokfnhpk9u/.repoyard/local_store/my_remote/01K9YZXGD12EFH12CAJZ028YKY__test_repo_2/data/.git/
Initialized empty Git repository in /private/tmp/sync_repokfnhpk9u/.repoyard/local_store/my_remote/01K9YZXGDMMF5J93B1GBZWDDSQ__test_repo_3/data/.git/
Initialized empty Git repository in /private/tmp/sync_repokfnhpk9u/.repoyard/local_store/my_remote/01K9YZXGE86N2KGX8A2BFAWQ72__test_repo_4/data/.git/
Initialized empty Git repository in /private/tmp/sync_repokfnhpk9u/.repoyard/local_store/my_remote/01K9YZXGEWVCXTD0F4AVGGF8BR__test_repo_5/data/.git/
Initialized empty Git repository in /private/tmp/sync_repokfnhpk9u/.repoyard/local_store/my_remote/0

Initialized empty Git repository in /private/tmp/sync_repokfnhpk9u/.repoyard/local_store/my_remote/01K9YZXGH8WDD08XZKVN66BMKZ__test_repo_9/data/.git/


Initialized empty Git repository in /private/tmp/sync_repokfnhpk9u/.repoyard/local_store/my_remote/01K9YZXGHWPBSS1WN0P45Q9BWZ__test_repo_10/data/.git/


Initialized empty Git repository in /private/tmp/sync_repokfnhpk9u/.repoyard/local_store/my_remote/01K9YZXGJF03ZBX8C41PK5MB5F__test_repo_11/data/.git/
Initialized empty Git repository in /private/tmp/sync_repokfnhpk9u/.repoyard/local_store/my_remote/01K9YZXGK39A7TGNV7YPG0TB30__test_repo_12/data/.git/
Initialized empty Git repository in /private/tmp/sync_repokfnhpk9u/.repoyard/local_store/my_remote/01K9YZXGKPH6WSSJ6T46PTGX9G__test_repo_13/data/.git/
Initialized empty Git repository in /private/tmp/sync_repokfnhpk9u/.repoyard/local_store/my_remote/01K9YZXGM7M46SS6W5G6FBX2E3__test_repo_14/data/.git/
Initialized empty Git repository in /private/tmp/sync_repokfnhpk9u/.repoyard/local_store/my_remote/01K9YZXGMST04YQJQ4GYHB8FJM__test_repo_15/data/.git/
Initialized empty Git repository in /private/tmp/sync_repokfnhpk9u/.repoyard/local_store/my_remote/01K9YZXGNAE1XX1XABS4HD51PY__test_repo_16/data/.git/
Initialized empty Git repository in /private/tmp/sync_repokfnhpk9u/.repoyard/local_store/my_re

# Exclude all repos using `exclude_repo`

In [ ]:
#|export
for repo_full_name in repo_full_names:
    exclude_repo(config_path=config_path, repo_full_name=repo_full_name)
    
# Verify that the repos have been excluded
repoyard_meta = get_repoyard_meta(config, force_create=True)
for repo_full_name in repo_full_names:
    assert not repoyard_meta.by_full_name[repo_full_name].check_included(config)

# Include all repos using `include_repo`

In [ ]:
#|export
for repo_full_name in repo_full_names:
    include_repo(config_path=config_path, repo_full_name=repo_full_name)
    
# Verify that the repos are included
repoyard_meta = get_repoyard_meta(config, force_create=True)
for repo_full_name in repo_full_names:
    assert repoyard_meta.by_full_name[repo_full_name].check_included(config)